In [ ]:
import array
import random
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import inspect

from deap import algorithms
from deap import base
from deap import creator
from deap import tools


data = (pd.read_csv("clean-data.csv"))

N = 11


PUNISHMENT = 140
COST_PUNISHMENT = 110
DUPLICATE_PUNISHMENT = 500

MAX_GKS = 1
MAX_DEF = 5
MAX_MID = 5
MAX_STR = 3


points = data['Points'] 
cost = data['Cost']
position = data['Position']

Postions = [['GK',1],['DEF',3],['MID',3],['STR',1]]

thispos = 'GK'
positionArray = []
for i in range(len(data)):
    if position[i] != thispos:
        positionArray.append(i)
    thispos = position[i]
#positionArray.append(len(data))

print(positionArray)

In [ ]:
def printTeam(team):
    thisTeam = data.iloc[team]
    thisTeam.loc["Total"] = thisTeam.sum(numeric_only=True, axis=0)
    print('Printed Team')
    print(thisTeam)

In [ ]:
def variAnd(population, toolbox, cxpb, mutpb):
    offspring = [toolbox.clone(ind) for ind in population]
    random.shuffle(offspring)
    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb:
            xMan, = toolbox.mutate(offspring[i])
            del xMan.fitness.values
            offspring.append(xMan)
    

    return offspring

In [ ]:
def eaGreekLetters(population, toolbox, cxpb, mutpb, ngen,
                   logbook = tools.Logbook(), mu=160, lambda_=240, stats=None, halloffame=None, verbose=__debug__):
    print("Using")
    print(inspect.stack()[0][3])
    print(inspect.stack()[1][3])
    logbook.header = ['gen', 'nevals', 'pop'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    #record = stats.compile(population) if stats is not None else {}
    #logbook.record(gen=0, nevals=len(invalid_ind), pop=len(population), **record)
    #if verbose:
        #print (logbook.stream)

    # Begin the generational process

    for gen in range(1, ngen + 1):
        # Vary the population
        

        #parents = tools.selection.selTournament(population, lambda_, 50)
        parents = toolbox.select(population)
        offspring = variAnd(parents, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        influx = toolbox.population(n = int(mu*0.5))
        new_invalid_ind = [ind for ind in influx if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, new_invalid_ind)
        for ind, fit in zip(new_invalid_ind, fitnesses):
            ind.fitness.values = fit
        # Select the next generation population
        

        # Update the statistics with the new population
        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), pop=len(population), **record)
        if verbose:
            print (logbook.stream)

        population = tools.selection.selBest(list(set(offspring + population)), int(mu*1.5)) + influx

    return population, logbook

In [ ]:
def playerMutator(individual, indpb):
    for i in individual:
        if random.random() < indpb:
            if i >= 466:
                i = random.randint(466, 522)
            else:
                i = random.randint(0, 465)

    return individual,

In [ ]:
def createIndividual(individual):
    team = []
    for pos in Postions:
        for value in (data[data["Position"] == pos[0]].sample(pos[1]).index):
            team.append(value)
            
    while(len(team)<N):
        x = random.randrange(1,4)
        team.append(data[data["Position"] == Postions[x][0]].sample().index[0])   
    
    #team.sort()
    
    ind = individual(team)
    
    return ind

def createInd():
    team = []
    for pos in Postions:
        for value in (data[data["Position"] == pos[0]].sample(pos[1]).index):
            team.append(value)
            
    while(len(team)<N):
        x = random.randrange(1,4)
        team.append(data[data["Position"] == Postions[x][0]].sample().index[0])   
    
    #team.sort()
    
    return team

printTeam(createInd())

def hash_ind_list(self):
    #self.sort()
    return hash(tuple(self))

In [ ]:
def fitness(individual):
    runningScore = 0
    runningCost = 0
    gks = 0
    defs = 0
    mids = 0
    strs = 0
    for gene in individual:
        runningScore += points[gene]
        runningCost += cost[gene]
        if position[gene][0] == 'G':
            gks += 1
            if gks > MAX_GKS:
                runningScore -= PUNISHMENT
    
        elif position[gene][0] == 'D':
            defs += 1
            if defs > MAX_DEF:
                runningScore -= PUNISHMENT
    
        elif position[gene][0] == 'M':
            mids += 1
            if mids > MAX_MID:
                runningScore -= PUNISHMENT
    
        elif position[gene][0] == 'S':
            strs += 1
            if strs > MAX_STR:
                runningScore -= PUNISHMENT
    
    if runningCost>100:
        x = (runningCost - 100) * COST_PUNISHMENT
        if x < COST_PUNISHMENT:
            x = COST_PUNISHMENT       
        runningScore -= x
        
    setLengthInd = len(set(individual))

    # if lentthInd != setLengthInd:
    #     noOfDuplicates = lentthInd - setLengthInd
    #     runningScore -= (noOfDuplicates * DUPLICATE_PUNISHMENT)

    if abs(setLengthInd-N) > 0: 
        runningScore -= (abs(setLengthInd-N) * DUPLICATE_PUNISHMENT)

    return runningScore,

In [ ]:
#[0, 180, 376, 466, 523]

def splitPos(ind):
    twoDArray = []
    for things in positionArray:
        twoDArray.append([])
    for i in ind:
        if i >= positionArray[3]:
            twoDArray[0].append(i)
        elif i >= positionArray[2]:
            twoDArray[3].append(i)
        elif i >= positionArray[1]:
            twoDArray[2].append(i)
        elif i >= positionArray[0]:
            twoDArray[1].append(i)


        # for j in range(len(positionArray)-1):
        #     if i >= positionArray[j]:
        #         if i < positionArray[j + 1]:
        #             twoDArray[j].append(i)
    return twoDArray


def sectionCrossover(p1, p2):
    
    random.shuffle(p1)
    random.shuffle(p2)

    minSize = min(len(p1), len(p2))
    
    c1 = p1
    c2 = p2
    if minSize == 0:
        print("minSizeiszero")

    if minSize == 1:
        c1[0], c2[0] = c2[0], c1[0]
    else:
        k = random.randint(1, minSize)
        for i in range(0, k):
            c1[i], c2[i] = c2[i], c1[i]

    return c1, c2


def positionCrossover(ind1, ind2, indpb = 0.5):
    crossoverPerformed = False
    p1 = splitPos(toolbox.clone(ind1))
    p2 = splitPos(toolbox.clone(ind2))
    c1 = p1
    c2 = p2

    
    for i in range(len(p1)):
        if (random.randint(0,100)/100) < indpb:
            c1[i], c2[i] = sectionCrossover(p1[i],p2[i])
            crossoverPerformed = True


    if crossoverPerformed!=True:
        i = random.randint(0,len(p1)-1)
        c1[i], c2[i] = sectionCrossover(p1[i],p2[i])
    
    child1 = [val for sublist in c1 for val in sublist]
    child2 = [val for sublist in c2 for val in sublist]

    for i in range(len(child1)):
        ind1[i], ind2[i] = child1[i], child2[i]

    return ind1, ind2

In [ ]:
NGEN = 150
MU, LAMBDA_ = 100,100 # MU * 2 = total number of population for each generation, LAMBDA_ = number of individuals selected for xOver and number of offspring produced
POPULATIONSIZE = 200

In [ ]:
toolbox = base.Toolbox()

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax, __hash__=hash_ind_list)



toolbox.register("individual", createIndividual, creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)

#toolbox.register("mate", tools.cxTwoPoint)
#toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mate", positionCrossover, indpb=0.8)

#toolbox.register("mutate", tools.mutUniformInt, indpb = 0.05, low = 0, up = len(data)-1)
toolbox.register("mutate", playerMutator, indpb = 0.075)

toolbox.register("select", tools.selRoulette, k=int(LAMBDA_))
#toolbox.register("select", tools.selTournament, k=int(LAMBDA_), tournsize=int(MU/3))

In [ ]:
s = "Using Toolbox:\n{0}\n{1}\n{2}\nfor {3} generations\nMU, LAMBDA_ = {4}, {5}\nPOPULATIONSIZE = {6}".format(toolbox.mate, toolbox.mutate, toolbox.select, NGEN, MU, LAMBDA_, POPULATIONSIZE)

In [ ]:
import time
def main():

    # choose a population size: e.g. 200

    pop = toolbox.population(n=POPULATIONSIZE)
    #pop.append(toolbox.newboy())
    
    # keep track of the single best solution found
    hof = tools.HallOfFame(1)
 
    # create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
    # to calculate the stats and label them with convenient labels
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)


    print (s)

    #int(random.randint(0,10)/10)
    
    # run the algorithm: we need to tell it what parameters to use
    # cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
    #pop, log = eaConfigured(pop, toolbox, cxpb=0.9, mutpb=0.1, ngen=600, stats=stats, halloffame=hof, verbose=True)
    pop, log = eaGreekLetters(pop, toolbox, mu=MU, lambda_=LAMBDA_, cxpb=1, mutpb=mutata, ngen=NGEN, stats=stats, halloffame=hof, verbose=__debug__)
    #pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=160, lambda_=240, cxpb=0.9, mutpb=0.1, ngen=600, stats=stats, halloffame=hof, verbose=True)
    #pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.9, mutpb=0.1, ngen=600, stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof


In [ ]:
pop, log, hof = main()

In [ ]:
best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# look in the logbook to see what generation this was found at

max = log.select("max")  # max fitness per generation stored in log

for i in range(NGEN):  # set to ngen
        fit = max[i]
        if fit == best:
            break        
        
print("max fitness found is %s at generation %s" % (best, i))


print(hof[0])

bestTeam = data.iloc[hof[0]]
bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)
    
print('best team')


print(bestTeam)

gen = log.select("gen")
fit_max = log.select("max")
fit_min = log.select("min")
fit_avg = log.select("avg")

fig, ax1 = plt.subplots()
line1 = ax1.plot(gen, fit_max, "b-", label="max Fitness", color="r")
line2 = ax1.plot(gen, fit_min, "b-", label="min Fitness", color="b")
line3 = ax1.plot(gen, fit_avg, "b-", label="avg Fitness", color="g")
ax1.set_xlabel("Generations")
ax1.set_ylabel("Fitness", color="b")
#plt.annotate(bestTeam.to_string)
#plt.figtext(0.5, 0.01, bestTeam.to_string, ha="left", fontsize=10, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
y = -0.55
plt.figtext(0.99, y, ("{0}\n{1}".format(bestTeam.to_string,hof[0])), horizontalalignment='right')
ax1.set_title("{0} \n max fitness found is {1} at generation {2}".format(s, best, i))

for tl in ax1.get_yticklabels():
    tl.set_color("b")

    
lns = line1+line2+line3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="center right")

In [ ]:
bestTeam = data.iloc[hof[0]]
bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)
    
print('best team')
print(bestTeam)

besters = [466, 0, 1, 2, 180, 181, 182, 183, 377, 376, 378]

bestTeam = data.iloc[besters]
bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)
    
print('best team')
print(bestTeam)

In [ ]:
best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# look in the logbook to see what generation this was found at

max = log.select("max")  # max fitness per generation stored in log

for i in range(200):  # set to ngen
        fit = max[i]
        if fit == best:
            break        
        
print("max fitness found is %s at generation %s" % (best, i))


print(hof[0])

bestTeam = data.iloc[hof[0]]
bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)

print('best team')
print(bestTeam)


gen = log.select("gen")
fit_max = log.select("max")
fit_min = log.select("min")
fit_avg = log.select("avg")

fig, ax1 = plt.subplots()
line1 = ax1.plot(gen, fit_max, "b-", label="max Fitness", color="r")
line2 = ax1.plot(gen, fit_min, "b-", label="min Fitness", color="b")
line3 = ax1.plot(gen, fit_avg, "b-", label="avg Fitness", color="g")
ax1.set_xlabel("Generations")
ax1.set_ylabel("Fitness", color="b")
ax1.set_title("{0}\nmax fitness found is {1} at generation {2}".format(s,best, i))
for tl in ax1.get_yticklabels():
    tl.set_color("b")

    
lns = line1+line2+line3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="center right")

In [ ]:
besters = [466, 0, 1, 2, 180, 181, 182, 183, 377, 376, 378]

teams = []

teams.append([466,0, 15, 138, 1, 163, 183, 185, 180, 187, 376])
teams.append([470, 1, 138, 15, 0, 23, 183, 196, 198, 180, 376])
teams.append([470, 1, 163, 23, 0, 15, 183, 196, 180, 198, 376])
teams.append([470, 15, 138, 1, 0, 23, 196, 198, 183, 180, 376])
teams.append([470, 15, 163, 0, 1, 23, 198, 183, 180, 196, 376])
teams.append([470, 138, 15, 23, 0, 1, 198, 196, 183, 180, 376])
teams.append([470, 0, 138, 15, 1, 23, 198, 183, 196, 180, 376])
teams.append([470, 15, 1, 0, 163, 23, 180, 196, 183, 198, 376])
teams.append([470, 15, 138, 0, 1, 23, 198, 180, 183, 196, 376])
teams.append([470, 15, 0, 1, 23, 163, 196, 198, 180, 183, 376])
teams.append([470, 1, 0, 138, 15, 23, 183, 180, 196, 198, 376])
teams.append([470, 1, 15, 0, 163, 23, 180, 196, 198, 183, 376])
teams.append([470, 1, 15, 0, 23, 163, 180, 183, 198, 196, 376])
teams.append([470, 15, 0, 1, 23, 138, 183, 196, 198, 180, 376])
teams.append([470, 163, 15, 0, 1, 23, 183, 196, 198, 180, 376])
teams.append([470, 1, 15, 138, 0, 23, 196, 183, 198, 180, 376])
teams.append([470, 138, 15, 1, 0, 23, 196, 180, 198, 183, 376])

bestTeam = data.iloc[besters]
bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)
    
print('best team')
print(bestTeam)


for ind in teams:
    bestTeam = data.iloc[ind]
    bestTeam.loc["Total"] = bestTeam.sum(numeric_only=True, axis=0)
        
    print('best team')
    print(bestTeam)

In [ ]:
import numpy as np
# HELPFUL DATA 
# these can be used for calculating points and costs and are also used in the constraint_checking function
points = data['Points'] 
cost = data['Cost']
num_players = len(data.index)  

# create lists with all elements initialised to 0
gk = np.zeros(num_players)
mid = np.zeros(num_players)
defe = np.zeros(num_players)
stri = np.zeros(num_players)

numberOfPostions = [0,0,0,0]

for i in range(num_players):
    if data['Position'][i] == 'GK':
        gk[i] = 1
        numberOfPostions[0] += 1 
    elif data['Position'][i] == 'DEF':
        defe[i] = 1
        numberOfPostions[1] += 1 
    elif data['Position'][i] == 'MID':
        mid[i] = 1
        numberOfPostions[2] += 1 
    elif data['Position'][i] == 'STR':
        stri[i]=1
        numberOfPostions[3] += 1 

print("dunzo")
print(numberOfPostions)

def check_constraints(individual):
     
    broken_constraints = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints+=1
        print("total players is %s " %(c1))
        
    
    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 > 100:
        broken_constraints+=1
        print("cost is %s " %(c2))
    
    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        print("goalies is %s " %(c3))
    
    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        print("DEFE is %s " %(c4))
            
    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        print("MID is %s " %(c5))
        
    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        print("STR is %s " %(c6))
        
    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    
    totalpoints = np.sum(np.multiply(points, individual))
        
        
    print("total broken constraints: %s" %(broken_constraints))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))
    
    return broken_constraints, totalpoints






def convertTeam(ind):
    team = numpy.zeros(len(data))
    for i in ind:
        team[i] = 1
    return team

check_constraints(convertTeam([470, 1, 0, 36, 30, 9, 196, 180, 372, 376, 383]))

check_constraints(convertTeam([469, 0, 41, 1, 151, 23, 183, 196, 185, 376, 383]))

check_constraints(convertTeam(hof[0]))

